In [1]:
from pytube import YouTube
import pandas as pd
import cv2
import re

In [2]:
links = [
    ('https://www.youtube.com/watch?v=BD_pjm7xvgg', 'movie'),
    ('https://www.youtube.com/watch?v=RlfbRpZvT70', 'movie'),
    ('https://www.youtube.com/watch?v=Zh-xHOtURxc', 'movie'),
    ('https://www.youtube.com/watch?v=S6EAJCpBku8', 'movie'),
    ('https://www.youtube.com/watch?v=EN-pRNfp8cA', 'movie'),
    ('https://www.youtube.com/watch?v=sfDcqfRiJZU&t=1245s', 'movie'),
    ('https://www.youtube.com/watch?v=7pjvh41e_1U&t=2003s', 'movie'),
    ('https://www.youtube.com/watch?v=mPYWVDOZxzw', 'movie'),
    ('https://www.youtube.com/watch?v=TAOBG5a_lV8', 'tour'),
    ('https://www.youtube.com/watch?v=Nde5P8Ob2xg', 'tour'),
]

In [4]:
download_files = True

if download_files:
    
    info = pd.DataFrame(columns=['id', 'Title', 'URL', 'Resolution', 'FPS', 'Size (MB)', 'Type'])

    for i, link in enumerate(links):
            
        vtype = link[1]
        link = link[0]
        
        yt = YouTube(link)

        stream = [x for x in yt.streams.filter(file_extension='mp4', progressive=False, only_video=True, res='480p') if 'avc1' in x.video_codec][0]

        stream.download(filename='{}.mp4'.format(i+1))

        info = info.append({
            'id':int(i+1),
            'Title':yt.title,
            'URL':link,
            'Resolution':stream.resolution,
            'FPS':stream.fps,
            'Size (MB)':stream.filesize/1024/1024,
            'Type':vtype
        }, ignore_index=True)

        print('{}/{}'.format(i+1, len(links)), flush=True, end='\r')
        
    info.to_csv('info.csv', index=False)
    
else:
    info = pd.read_csv('info.csv')

Take first 50 minutes per each movie. Skip the first 5 minutes

In [ ]:
# For movies
for i, x in info.iterrows():
    
    file_name = '{}.mp4'.format(x['id'])
    fps = x['FPS']
        
    cap = cv2.VideoCapture(file_name)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    curr_frame = fps * 60 * 5
    
    num_images = 0
    
    while(True):
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, curr_frame)
        ret, frame = cap.read()
        
        if not ret or curr_frame >= int(length * 0.35):
            break
            
#         if curr_frame > fps * 60 * 50 and x['Type']=='movie':
#             break
        
        num_images += 1
        
        if x['Type'] == 'tour':
            curr_frame += fps * 60 # fastforward 60 seconds
        else:
            curr_frame += fps * 5 # fastforward 5 seconds
        
    print(num_images)